# Analyzing Chicago Crime - Extracting data

link: https://dev.socrata.com/foundry/data.cityofchicago.org/x2n5-8w5q

## To do

- [x] sign up for app token [here](https://data.cityofchicago.org/login)
- [x] create env file in main dir + Update jupyter notebook to use env file
- [ ] Push to github
- [ ] Recheck deduplicate for-loop (confirm we can dedup along 'case_' column)
- [ ] Create logic for extraction, using the while loop and pagination in tandem with **order by `:id`**
- [ ] Transformations
- [ ] AWS
- [ ] Database
- [ ] Logic for UPSERT
- [ ] Docker

## Questions

- [ ] How will we manage the env file in the docker container?
- [ ] Who's App token are we going to use? Or are we to assume that the person using our container has to get their own app token?

## Imports & configurations

In [8]:
# imports
import pandas as pd
import requests
import os
from dotenv import load_dotenv 
import json

# configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# .env
dotenv_path = os.path.expanduser("~/Documents/DEC/dec-proj1-chicago-crime/.env")  #Enter path to env file here
load_dotenv(dotenv_path)

# variables
APP_TOKEN = os.environ.get("APP_TOKEN")

## Extracting Logic (WIP)

- retrieve MAX date_of_occurence from database
- Round up to next day (2024-01-01T23:50:00.000 --> 2024-01-02T00:00:00.000), this becomes `start_time` variable
- Add 23 hours, 59 mins, 59 seconds, 999 milliseconds to the time above using python datetime module. This become `end_time` variable
- Query api using the 'BETWEEN .. AND ..' function using the `start_time` and `end_time` variables
- Do transformations in python
- upsert into database

### Problems

- we really won't be upserting, since we're only going to be uploading a day's worth of data at a time.

### Thoughts from a balcony

- If we upload only a day's worth of data at a time, every hour, that is going to be:
  - 365 - 7 (1 year minus most recent 7 days) = **358** days in dataset
  - 358 uploads / 24 uploads per day = Approx **15 days** to get all the data.
- If we upload 7 days worth of data at a time, every hour...
  - 358 / 7 = 52 uploads
  - 52 / 24 = **2.5 days** to get all the data
  - 7 days worth of data is ~5000 records per upload
- So... we could upload 7 days worth of data at a time, and once we start getting no records, meaning we have reached the end of the dataset, we can fetch the min date from the dataset, and then start fetching 7 days worth of data again. This will be upserting then.

## Extracting Data (WIP)

Will update this once done completing logic using `requests` library

## Transformations (WIP)

### Drop 'location' column 

drop 'location' column since it displays data we already have under the longitude and latitude columns

In [ ]:
df.drop('location', axis=1, inplace=True)

### Deduplicate (WIP)

I Need re-run the for loop below, there was an error in the logic! Load saved data from csv.

## Checking for Duplicates

In [ ]:
duplicates_all_cols = df[df.duplicated()]

len(duplicates_all_cols)

There are still duplicated records when filtering by case number, indicating that these cases have multiple records where one of the other columns have unique values.

In [ ]:
duplicates = df[df.duplicated(subset='case_')]

len(duplicates)

**So Questions:**

1. What columns have multiple unique values for cases with multiple records?
2. And can we drop duplicates based on just the case number?

In [ ]:
# view cases that have duplicate records
duplicates = duplicates.sort_values('case_')

duplicates

Below, checking which columns have multiple unique values for each case number with duplicate records:

In [ ]:
# create list of cols
col_list = list()
for col in duplicates.columns:
    col_list.append(col)


# initialize empty list to capture case numbers and what columns are different
duplicates_breakdown = list()


# Check what is duplicated
for case in duplicates['case_']:  # cycle through cases that have duplicated records
    duplicates_cols = list()  # initialize empty list to capture all columns that have different values within a set of duplicated records
    for col in col_list:  # cycle through columns
        if len((df[df['case_'] == case][col]).unique()) > 1:
            duplicates_cols.append(col)
    duplicates_breakdown.append(f'{case}, {duplicates_cols}')

duplicates_breakdown

In [ ]:
len(duplicates_breakdown)

**Answering questions from above:**

1.
2.

## Using Requests to extract

Notes:

- For pagination, **order by `:id`** (since there are multiple crime cases with the same date_of_occurrence), and use the `offset` and `limit` param in tandem. Increase offset by the value of the limit to grab the next page. Ex:

  ```md
  limit = 10
  
  pg 1
  offset = 0

  pg 2:
  offset = 0 + 10 = 10

  pg 3
  offset = 10 + 10 = 20
  ```

- We can use a while loop, where we query the api as long as the len(data) we're receiving is equal to the limit value, and with each pass through increase offset by the limit value to get the next pg of data, until we reach the end of the dataset

Testing Pagination below:

In [ ]:
# df1 (pg1)

import requests

start_date = '2023-11-06T00:00:00.000'

end_date = '2023-11-19T23:59:59.999'

soql_date = f"$where=date_of_occurrence between '{start_date}' and '{end_date}'"

limit = 1000

offset = 0

response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                        f"$$app_token={APP_TOKEN}&"
                        f"$order=:id"  #a date of occurrence can have multiple cases, so better to use :id since that will lock the sequence of records.
                        f"&{soql_date}"
                        f"&$limit={limit}"
                        f"&$offset={offset}")

# print the message
data1 = response.json()
print(data1)
assert response.status_code == 200

In [ ]:
df1 = pd.json_normalize(data1)

print(len(df1))
df1

In [ ]:
print(df1['case_'])

In [ ]:
# df2 (pg2)

offset = 0+1000  #pg2

response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                        f"$$app_token={APP_TOKEN}&"
                        f"$order=:id"
                        f"&{soql_date}"
                        f"&$limit={limit}"
                        f"&$offset={offset}")

# print the message
data2 = response.json()
print(data2)
assert response.status_code == 200

In [ ]:
df2 = pd.json_normalize(data2)

print(len(df2))
df2

In [ ]:
print(df2['case_'])

In [ ]:
# df3 (pg1 + pg2)

full_limit = 1000 + 1000

response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                        f"$$app_token={APP_TOKEN}&"
                        f"$order=:id"
                        f"&{soql_date}"
                        f"&$limit={full_limit}"
                        )

# print the message
data3 = response.json()
print(data3)
assert response.status_code == 200

In [ ]:
df3 = pd.json_normalize(data3)

print(len(df3))
df3

In [ ]:
print(df3['case_'])

Testing querying api for MINIMUM date_of_occurrence from dataset - **Success**

In [ ]:
# min date_of_occurence in dataset

response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                        f"$$app_token={APP_TOKEN}&"
                        f"&$select=min(date_of_occurrence)"
                        )

# print the message
data3 = response.json()
print(data3)
assert response.status_code == 200